In [43]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression, SGDRegressor, ElasticNet, RidgeCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import xgboost as xgb
from pylab import *

import numpy as np
from numpy import linalg as LA
import pandas as pd
import os

# Data

In [56]:
def DPMU(phase,dpmu_data,phase_letter,letter):
    for (columnName, columnData) in phase.iteritems():
        i=0
        if 'Meter' in columnName:
            split = columnName.split('Meter')
            house_name = f'Meter{letter}_'+ split[1]
            i+=1
            dpmu_values = pd.Series(columnData.values).str.replace("i", "j").apply(lambda x: abs(complex(x)))
            dpmu_data[house_name] = dpmu_values
        else:
            split_column_names = columnName.split('_')
            house_name = 'house'+str(split_column_names[phase_letter])
            dpmu_values = pd.Series(columnData.values).str.replace("i", "j").apply(lambda x: abs(complex(x)))
            dpmu_data[house_name] = dpmu_values
    return dpmu_data

In [57]:
def Input(month,input_player_df):
    directory = fr'C:\Users\aniss\grid-edge\data\summer\input_csv\{month}'

    for filename in os.listdir(directory):
        if str(filename).startswith('25.csv'):
            continue
        player = pd.read_csv(os.path.join(directory, filename))
        player['grid'] = player['grid'].str.replace("i", "j").apply(lambda x: complex(x))
        node_id = player['id'].loc[0]
        column_name_p = 'p_'+str(node_id)
        column_name_q = 'q_'+str(node_id)
    #change .head() value to target dataframe row number
        input_player_df[column_name_p] = player['grid'].apply(lambda x: x.real).head(44640)
        input_player_df[column_name_q] = player['grid'].apply(lambda x: x.imag).head(44640)
    return input_player_df

In [58]:
dpmu_phaseA_aug = pd.read_csv(r'C:\Users\aniss\grid-edge\data\summer\targets\trimmed_targets\aug\multi_voltageA.csv')
dpmu_phaseB_aug = pd.read_csv(r'C:\Users\aniss\grid-edge\data\summer\targets\trimmed_targets\aug\multi_voltageB.csv')
dpmu_phaseC_aug = pd.read_csv(r'C:\Users\aniss\grid-edge\data\summer\targets\trimmed_targets\aug\multi_voltageC.csv')

dpmu_phaseA_july = pd.read_csv(r'C:\Users\aniss\grid-edge\data\summer\targets\trimmed_targets\july\multi_voltageA.csv')
dpmu_phaseB_july = pd.read_csv(r'C:\Users\aniss\grid-edge\data\summer\targets\trimmed_targets\july\multi_voltageB.csv')
dpmu_phaseC_july = pd.read_csv(r'C:\Users\aniss\grid-edge\data\summer\targets\trimmed_targets\july\multi_voltageC.csv')

dpmu_phaseA_june = pd.read_csv(r'C:\Users\aniss\grid-edge\data\summer\targets\trimmed_targets\june\multi_voltageA.csv')
dpmu_phaseB_june = pd.read_csv(r'C:\Users\aniss\grid-edge\data\summer\targets\trimmed_targets\june\multi_voltageB.csv')
dpmu_phaseC_june = pd.read_csv(r'C:\Users\aniss\grid-edge\data\summer\targets\trimmed_targets\june\multi_voltageC.csv')

dpmu_phaseA_sept = pd.read_csv(r'C:\Users\aniss\grid-edge\data\summer\targets\trimmed_targets\sept\multi_voltageA.csv')
dpmu_phaseB_sept = pd.read_csv(r'C:\Users\aniss\grid-edge\data\summer\targets\trimmed_targets\sept\multi_voltageB.csv')
dpmu_phaseC_sept = pd.read_csv(r'C:\Users\aniss\grid-edge\data\summer\targets\trimmed_targets\sept\multi_voltageC.csv')

In [59]:
dpmu_phaseA_aug = dpmu_phaseA_aug.drop(['Meter5', '# timestamp'], axis=1)
dpmu_phaseB_aug = dpmu_phaseB_aug.drop(['Meter5', '# timestamp'], axis=1)
dpmu_phaseC_aug = dpmu_phaseC_aug.drop(['Meter5', '# timestamp'], axis=1)

dpmu_phaseA_july = dpmu_phaseA_july.drop(['Meter5', '# timestamp'], axis=1)
dpmu_phaseB_july = dpmu_phaseB_july.drop(['Meter5', '# timestamp'], axis=1)
dpmu_phaseC_july = dpmu_phaseC_july.drop(['Meter5', '# timestamp'], axis=1)

dpmu_phaseA_june = dpmu_phaseA_june.drop(['Meter5', '# timestamp'], axis=1)
dpmu_phaseB_june = dpmu_phaseB_june.drop(['Meter5', '# timestamp'], axis=1)
dpmu_phaseC_june = dpmu_phaseC_june.drop(['Meter5', '# timestamp'], axis=1)

dpmu_phaseA_sept = dpmu_phaseA_sept.drop(['Meter5', '# timestamp'], axis=1)
dpmu_phaseB_sept = dpmu_phaseB_sept.drop(['Meter5', '# timestamp'], axis=1)
dpmu_phaseC_sept = dpmu_phaseC_sept.drop(['Meter5', '# timestamp'], axis=1)

In [60]:
phaseA = 0
phaseB = 1
phaseC = 2
A = "A"
B = "B"
C = "C"

In [61]:
dpmu_data_aug = pd.DataFrame()
dpmu_data_july = pd.DataFrame()
dpmu_data_june = pd.DataFrame()
dpmu_data_sept = pd.DataFrame()

In [62]:
dpmu_data_aug = DPMU(dpmu_phaseA_aug,dpmu_data_aug,phaseA, A)
dpmu_data_aug = DPMU(dpmu_phaseB_aug,dpmu_data_aug, phaseB, B)
dpmu_data_aug = DPMU(dpmu_phaseC_aug,dpmu_data_aug, phaseC, C)

dpmu_data_july = DPMU(dpmu_phaseA_aug,dpmu_data_july,phaseA, A)
dpmu_data_july = DPMU(dpmu_phaseB_aug,dpmu_data_july, phaseB, B)
dpmu_data_july = DPMU(dpmu_phaseC_aug,dpmu_data_july, phaseC, C)

dpmu_data_june = DPMU(dpmu_phaseA_aug,dpmu_data_june,phaseA, A)
dpmu_data_june = DPMU(dpmu_phaseB_aug,dpmu_data_june, phaseB, B)
dpmu_data_june = DPMU(dpmu_phaseC_aug,dpmu_data_june, phaseC, C)

dpmu_data_sept = DPMU(dpmu_phaseA_aug,dpmu_data_sept,phaseA, A)
dpmu_data_sept = DPMU(dpmu_phaseB_aug,dpmu_data_sept, phaseB, B)
dpmu_data_sept = DPMU(dpmu_phaseC_aug,dpmu_data_sept, phaseC, C)

In [74]:
dpmu_data = pd.DataFrame()
dpmu_data = pd.concat([dpmu_data_june, dpmu_data_july,dpmu_data_aug,dpmu_data_sept], ignore_index = True)

In [64]:
dpmu_data_2 = dpmu_data[0:175680]

In [65]:
aug = "aug"
july = "july"
june = "june"
sept = "sept"

In [66]:
input_player_df_aug = pd.DataFrame()
input_player_df_july = pd.DataFrame()
input_player_df_june = pd.DataFrame()
input_player_df_sept = pd.DataFrame()

In [67]:
input_player_df_aug = Input(aug,input_player_df_aug)
input_player_df_july = Input(july,input_player_df_july)
input_player_df_june = Input(june,input_player_df_june)
input_player_df_sept = Input(sept,input_player_df_sept)

In [68]:
input_player_df = pd.DataFrame()
input_player_df = pd.concat([input_player_df_june,input_player_df_july,input_player_df_aug,input_player_df_sept], ignore_index=True)

# Model

In [75]:
# get the dataset
def get_dataset():
    x, y = input_player_df.copy().to_numpy(), dpmu_data_2.copy().to_numpy()
    x_train_split, x_test_split, y_train_split, y_test_split = train_test_split(x, y, test_size=0.33, random_state=42) 
    return x_train_split, x_test_split, y_train_split, y_test_split

In [70]:
# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mse', optimizer='adam')
    return model

In [71]:
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(x, y):
    results = list()
    n_inputs, n_outputs = x.shape[1], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(x):
        # prepare data
        x_train, x_test = x[train_ix], x[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        model = get_model(n_inputs, n_outputs)
        # fit model
        model.fit(x_train, y_train, verbose=1, epochs=100)
        # evaluate model on test set
        #mse = model.evaluate(x_test, y_test, verbose=0)
        
        y_pred = model.predict(x_test)
        names = y_test.columns
        df_y_pred = pd.DataFrame(y_pred,columns = names)
        scaling_all = pd.concat([y_test,df_y_pred], keys=['y_test','y_pred'])
        
        scaler = StandardScaler()
        scaler.fit(scaling_all)
        scaledDF = pd.DataFrame(scaler.transform(scaling_all),columns = names)
        
        scaled_y_pred = scaledDF[:57975]
        scaled_y_test = scaledDF[57975:]
        
        L2_ratios = []
        for index, row in scaled_y_test.iterrows():
            L2_ratios.append(LA.norm(row))
        
        #dividing all y_pred by the L2 ratios:
        ratio_index = 0
        scaled_ratio_y_pred = []
        for index, row in scaled_y_pred.iterrows():
            scaled_ratio_y_pred.append(row/L2_ratios[ratio_index])
            ratio_index += 1
        
        # store result
        print(mse)
        results.append(mse)
        return scaled_ratio_y_pred

In [72]:
# train a model using repeated k-fold cross-validation
def train_model(x_train_split, y_train_split):
    results = list()
    n_inputs, n_outputs = x_train_split.shape[1], y_train_split.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(x_train_split):
        # prepare data
        x_train, x_test = x[train_ix], x[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        model = get_model(n_inputs, n_outputs)
        # fit model
        model.fit(x_train, y_train, verbose=1, epochs=100)
        return model

def evaluation_model(model, x_test_split, y_test_split):
    y_pred = model.predict(x_test_split)
    names = y_test_split.columns
    df_y_pred = pd.DataFrame(y_pred,columns = names)
    scaling_all = pd.concat([y_test_split,df_y_pred], keys=['y_test','y_pred'])

    scaler = StandardScaler()
    scaler.fit(scaling_all)
    scaledDF = pd.DataFrame(scaler.transform(scaling_all),columns = names)

    scaled_y_pred = scaledDF[:57975]
    scaled_y_test = scaledDF[57975:]

    L2_ratios = []
    for index, row in scaled_y_test.iterrows():
        L2_ratios.append(LA.norm(row))

    #dividing all y_pred by the L2 ratios:
    ratio_index = 0
    scaled_ratio_y_pred = []
    for index, row in scaled_y_pred.iterrows():
        scaled_ratio_y_pred.append(row/L2_ratios[ratio_index])
        ratio_index += 1

    # store result
    print(mse)
    results.append(mse)
    return scaled_ratio_y_pred, scaled_y_test


In [ ]:
# load dataset
x_train_split, x_test_split, y_train_split, y_test_split = get_dataset()
# evaluate model
model_trained = train_model(x, y)
scaled_ratio_y_pred, scaled_y_test = evaluation_model(model_trained, x_test_split, y_test_split)
# summarize performance
#print('MSE: %.3f (%.3f)' % (mean(results), std(results)))
print('MSE: %.3f (%.3f)' % (scaled_y_test, scaled_ratio_y_pred))

Epoch 1/100
4941/4941 [==============================] - 7s 1ms/step - loss: 16620393.3973
Epoch 2/100
4941/4941 [==============================] - 7s 1ms/step - loss: 443676.3229
Epoch 3/100
4941/4941 [==============================] - 7s 1ms/step - loss: 290274.7830
Epoch 4/100
4941/4941 [==============================] - 7s 1ms/step - loss: 241444.7819
Epoch 5/100
4941/4941 [==============================] - 7s 1ms/step - loss: 226184.9319
Epoch 6/100
4941/4941 [==============================] - 7s 1ms/step - loss: 219673.2132
Epoch 7/100
4941/4941 [==============================] - 6s 1ms/step - loss: 211221.5199
Epoch 8/100
4941/4941 [==============================] - 5s 1ms/step - loss: 207547.4763
Epoch 9/100
4941/4941 [==============================] - 6s 1ms/step - loss: 207863.3302
Epoch 10/100
4941/4941 [==============================] - 6s 1ms/step - loss: 204919.3757
Epoch 11/100
4941/4941 [==============================] - 6s 1ms/step - loss: 202794.4306
Epoch 12/100
4941